# Simulation

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

random_seed = 0

In [ ]:
def ind_max(temp):
    m = max(temp)
    list_m = list(filter(lambda x: temp[x] == m, range(len(temp))))
    return random.choice(list_m)

In [ ]:
def test_algorithm(algo,num_sims,T):
    chosen_arms = [0.0 for i in range(num_sims * T)] 
    rewards = [0.0 for i in range(num_sims * T)] 
    regrets = [0.0 for i in range(num_sims * T)]
    cumulative_rewards = [0.0 for i in range(num_sims * T)] 
    cumulative_regrets = [0.0 for i in range(num_sims * T)] 
    
    best_find = [0.0 for i in range(num_sims * T)] # if find the best -> 1 else -> 0
    cumulative_best_find = [0.0 for i in range(num_sims * T)]
    
    sim_nums = [0.0 for i in range(num_sims * T)] 
    times = [0.0 for i in range(num_sims * T)] 
    
    for sim in range(num_sims):
        k = 2
        p_list = [random.random() for i in range(k)]
        arms = [BernoulliArm(p) for p in p_list]
        best_arm = ind_max([arm.p for arm in arms]) # Index of the best arm.
        best_arm_reward = max([arm.p for arm in arms])

        n_arms = len(arms) # k
        sim = sim + 1 
        algo.initialize(n_arms)
        
        for t in range(T): # pull arm
            t = t + 1
            index = (sim - 1) * T + t - 1
            sim_nums[index] = sim
            times[index] = t 
      
            chosen_arm = algo.select_arm()
            chosen_arms[index] = chosen_arm
            
            if int(chosen_arm) == int(best_arm):
                best_find[index] = 1  
                if t == 1:
                    cumulative_best_find[index] = 1
                else :
                    cumulative_best_find[index] = cumulative_best_find[index - 1] + best_find[index]
            else:
                best_find[index] = 0
                if t == 1:
                    cumulative_best_find[index] = 0
                else :
                    cumulative_best_find[index] = cumulative_best_find[index - 1] + best_find[index]

            reward = arms[chosen_arms[index]].draw()
            regret = best_arm_reward - reward
            rewards[index] = reward
            regrets[index] = regret
      
            if t == 1:
                cumulative_rewards[index] = reward
                cumulative_regrets[index] = regret
            else:
                cumulative_rewards[index] = cumulative_rewards[index - 1] + reward
                cumulative_regrets[index] = cumulative_regrets[index - 1] + regret
      
            algo.update(chosen_arm, reward)
  
    temp_result =  [sim_nums, times, chosen_arms, rewards, cumulative_rewards, regrets,cumulative_regrets,best_find,cumulative_best_find]
    result = pd.DataFrame(temp_result).transpose()
    result.columns = ["Sim", "T", "ChosenArm", "Reward", "CumulativeReward", "Regret", "CumulativeRegret","best_find","cumulative_best_find"]

    return result